# Title: CORRELATION MODEL IN THE ADOPTION OF E-PAYMENT SERVICES

## Load Libraries

In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, KFold
from scipy.stats import pearsonr, spearmanr, kendalltau, pointbiserialr
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef

from xgboost import XGBClassifier

%matplotlib inline

## Load Custom Made Libraries

In [2]:
from Utilities.CFS import *
from Utilities.accuracy import *
from Utilities.corr_matrix import *
from Utilities.forward_selection import *
from Utilities.backward_elimination import *

from Visualization.model_graph import *
from Visualization.network_graph import *

## Load Dataset

In [3]:
df = pd.read_csv("Dataset/E-payment Cryptocurrency Coin.csv")

## Label Binarizer

In [4]:
def convert_nominal(arr, term_arr):
    tmp_dict = {val:ind for (ind, val) in enumerate(term_arr)}
    return arr.map(lambda x : tmp_dict[x])

## Split Dataset

In [5]:
mod_fac_df = pd.DataFrame()

column_arr = ["Age", "Gender", "Marital Status", "Education Level", "Work Industry", "Work Position"]

for (ind, col_name) in enumerate(column_arr):
    mod_fac_df[col_name] = df.iloc[:, 6 + ind]
    
# Replace Values in Work Industry
mod_fac_df = mod_fac_df.replace("Baking / Finance", "Banking / Finance")

# Label Binarize all columns
# Age
mod_fac_df["Age"] = convert_nominal(mod_fac_df["Age"], ["< 25 years", "26 - 40 years", "41 - 55 years", "above 55 years"])

# Gender
mod_fac_df["Gender"] = convert_nominal(mod_fac_df["Gender"], ["Male", "Female"])

# Marital Status
mod_fac_df["Marital Status"] = convert_nominal(mod_fac_df["Marital Status"], ["Single", "Married", "Other"])

# Education Level
mod_fac_df["Education Level"] = convert_nominal(mod_fac_df["Education Level"], ['Primary school', 'Secondary/High school', 'College/university', 'Graduate school', 'Other'])

# Work Industry
mod_fac_df["Work Industry"] = convert_nominal(mod_fac_df["Work Industry"], ['Banking / Finance', 'Education', 'Healthcare', 'Manufacturing', 'Retail / Hypermarket', 'Other'])

# Work Position
mod_fac_df["Work Position"] = convert_nominal(mod_fac_df["Work Position"], ['Junior management', 'Middle management', 'Top management', 'Professional', 'Other'])


In [6]:
utaut_fac_df = pd.DataFrame()

column_arr = df.iloc[:, 28:].columns
column_arr = [col.split(": ")[0] for col in column_arr]

for (ind, col_name) in enumerate(column_arr):
    utaut_fac_df[col_name] = df.iloc[:, 28 + ind]
    
# Change Data Type to int
utaut_fac_df = utaut_fac_df.astype(int)

## Create New CSV File

In [27]:
output_df = pd.DataFrame()

tmp_df = df.iloc[:, [17, 18]]

# Replace Values 
tmp_df = tmp_df.replace("No", 0)
tmp_df = tmp_df.replace("Yes", 1)

# Convert to INT
tmp_df = tmp_df.astype(int)

# Change Columns Name
tmp_df.columns = [col.split("[")[1][4:-1] for col in tmp_df.columns.tolist()]

arr_df = [mod_fac_df, utaut_fac_df, tmp_df]
output_df = pd.concat(arr_df, axis = 1)

In [28]:
output_df

,Age,Gender,Marital Status,Education Level,Work Industry,Work Position,PE1,PE2,PE3,PE4,...,T1,T2,T3,T4,BI1,BI2,BI3,BI4,Have you ever purchased anything using the E-payment mode?,"In the next six months, do you plan to purchase anything using the E-payment mode?"
0,0,1,0,2,0,4,2,2,2,2,...,2,1,2,3,3,3,3,3,1,1
1,0,1,0,2,5,4,1,1,1,1,...,2,2,2,2,2,2,2,2,1,1
2,2,1,0,2,3,1,4,5,4,5,...,3,3,3,4,3,4,4,4,1,1
3,0,0,0,2,1,4,3,3,3,3,...,3,3,3,3,4,4,3,3,0,1
4,0,1,0,2,5,4,3,3,4,4,...,3,4,4,4,4,5,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2,0,1,1,5,4,3,3,3,3,...,3,3,3,3,3,3,3,3,1,1
282,3,0,1,3,1,2,3,3,3,3,...,3,3,3,3,3,3,3,3,1,1
283,3,1,1,2,5,4,3,3,3,3,...,3,3,3,3,3,3,3,3,1,1
284,2,0,1,3,1,3,4,4,4,4,...,4,4,4,4,4,4,4,4,1,1


In [29]:
output_df.to_csv("for_weka.csv", index=False)